<a href="https://colab.research.google.com/github/AI-Front/ChatBots/blob/master/6-Empathic_Reactions_and_Humanization/Sentiment_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Out-of-the-box sentiment classifier with TextBlob

This tutorial describes how to use a simple sentiment classifier within [TextBlob](https://textblob.readthedocs.io/en/dev/advanced_usage.html#sentiment-analyzers) python library framework. 



In this tutorial we will go through

0. Installation and interaction
0. Simple skill examples you can make for the bot


![textblob.png](https://github.com/AI-Front/ChatBots/blob/master/6-Empathic_Reactions_and_Humanization/textblob.png?raw=1)

# Installation and interaction

We will now install TextBlob library itself and also download the nltk library dependent modules

In [1]:
!pip install textblob

In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

You can import 2 sentiment models eigher based on Naive Bayes Classifier or pattern-based one (default) 

The textblob.sentiments module contains two sentiment analysis implementations, PatternAnalyzer (based on the [pattern](https://www.clips.uantwerpen.be/pattern) library) and NaiveBayesAnalyzer (an NLTK classifier trained on a movie reviews corpus).

In [3]:
#default pattern-based way

from textblob import TextBlob

blob = TextBlob("I love building bots")
print(blob.sentiment)


Sentiment(polarity=0.5, subjectivity=0.6)


In [10]:
#NaiveBayes implementation

from textblob.sentiments import NaiveBayesAnalyzer

blob = TextBlob("I love chatbots!", analyzer=NaiveBayesAnalyzer())
print(blob.sentiment)

Sentiment(classification='pos', p_pos=0.5441400304414004, p_neg=0.45585996955859964)


# Simple skill examples

Sentiment classification itself can be a feature for training a more advanced dialog system or a manager, a ranging model, etc.

Still we can use it as a straitforward way to 
1. avoid negative messages from our own bot (this also applies for the [insult classifier](https://github.com/AI-Front/ChatBots/blob/master/6-Empathic_Reactions_and_Humanization/Insult_classifier.ipynb))
2. react to negative messages from the user (eg he is depressed or heartbroken)
3. add corresponding emojies to a message from the bot
4. react with emojies only to positive user messages (sometimes, instead of a chitchat, to model a human-like behaviour) 

## Example 1 (negative message from user)

**Important:** To prevent overreacting of the bot we can limit the skill activation by prediction probability of the classifier (for example, only for strongly negative comments with p_neg=0.8 and more)

**Important 2:** Sometimes it is better to switch the topic when the conversation is going in a negative way. You can add multimedia and links to your reaction messages so switch the attention and cheer up the user. Consider the platform and the usage of the bot (mobile vs desktop, screen size, etc)


In [4]:
import random

reply = ["Hey, what do you listen to when you feel sad? I love relaxing music: https://www.youtube.com/watch?v=5qap5aO4i9A", "Understandable 🥺 What can you do about it?", "I would feel the same if i were human",\
         '''What if I find you an emoji art?
         📲😳 I'm busy!💫
          ☝️👕
            🚽 💩💩💩💩''',
                  '''⁣     🚿￣＼
               😓   ｜
              ⁣/|\💧 ｜
          | 💧/ \   |
          ￣￣￣￣￣￣''',
         '''💐💐😉😊💐💐
          ☕ Cheer Up  🍵
          🍂 ✨ )) ✨  🍂
          🍂┃ (( * ┣┓ 🍂
          🍂┃*💗 ┣┛ 🍂
          🍂┗━━┛  🍂
          🎂 For YOU  🍰
          💐💐😌😚💐💐''']

In [5]:
print(random.choice(reply))

⁣     🚿￣＼
               😓   ｜
              ⁣/|\💧 ｜
          | 💧/ \   |
          ￣￣￣￣￣￣


In [29]:
class NegativeSkill():
    def __init__(self, *args, **kwargs):
        super(NegativeSkill, self).__init__(*args, **kwargs)
        self.reply = reply
 
    def __call__(self, text, states=None, **kwargs):
        res = self.get_answer(states)
        return res

    def fit(self, *args, **kwargs):
        """Fit skill function."""
        raise NotImplemented


    def classify(self, text, states):
        sentiment, p = self.check_skill(text)
        if sentiment:
            res = self.get_answer(p, states)
            return {'name': 'NegativeSkill', 'answer': res}
        return {}
  
    def get_answer(self, p=1, states=None):
        #this is skill confidence score - as we use classifier, we will use its class proability score
        score = p
        res = {
                "res_utterances": [[random.choice(self.reply)]],
                "states": states,
                "confidence_batch": [[score]]}
        return res


    @staticmethod
    def check_skill(text):
        #you can eigher put insult classifier here
        blob = TextBlob(text)
        if blob.sentiment.polarity < 0:
            return 'negative', blob.sentiment.subjectivity
        return False, 0

In [33]:
skill = NegativeSkill()
text_message = 'I hate it today'
skill.classify(text_message, {})

{'answer': {'confidence_batch': [[0.9]],
  'res_utterances': [['💐💐😉😊💐💐\n          ☕ Cheer Up  🍵\n          🍂 ✨ )) ✨  🍂\n          🍂┃ (( * ┣┓ 🍂\n          🍂┃*💗 ┣┛ 🍂\n          🍂┗━━┛  🍂\n          🎂 For YOU  🍰\n          💐💐😌😚💐💐']],
  'states': {}},
 'name': 'NegativeSkill'}

## Example 2 (emoji reactions)


In [24]:
class EmojiSkill():
    def __init__(self, *args, **kwargs):
        super(EmojiSkill, self).__init__(*args, **kwargs)
        self.sad = '😳 😞 😟 😠 😡 😬 😣 😖 😫 😩 😤 😮 😱 😨 😰 😯 😦 😧 😢 😥 😪 🤤 😓 😭 😵'.split()
        self.happy = '😀 😃 😄 😁 😆 😅 😂 🤣 😇 😉 😊 🙂 🙃 😋 😌 😍 😘 😗 😙 🤪 😜 😝 😛 🤑 😎 🤓 🤠 🥳 🤗 🤡 😏 🤩'.split()
 
    def __call__(self, text, states=None, **kwargs):
        res = self.get_answer(states)
        return res

    def fit(self, *args, **kwargs):
        """Fit skill function."""
        raise NotImplemented


    def classify(self, text, states):
        sentiment, p = self.check_skill(text)
        if sentiment!='neutral':
            res = self.get_answer(sentiment, p, states)
            return {'name': 'EmojiSkill', 'answer': res}
        return {}
  
    def get_answer(self,sentiment='positive', p=1, states=None):
        #this is the answer we want the bot to return
        answer_dic = {'positive':random.choice(self.happy), 'negative':random.choice(self.sad)}
        #this is skill confidence score - as we use classifier, we will use its class proability score
        score = p
        res = {
                "res_utterances": [[answer_dic[sentiment]]],
                "states": states,
                "confidence_batch": [[score]]}
        return res


    @staticmethod
    def check_skill(text):
        blob = TextBlob(text)
        if blob.sentiment.polarity > 0.49:
            return 'positive',blob.sentiment.subjectivity
        elif blob.sentiment.polarity < 0:
            return 'negative',blob.sentiment.subjectivity
        return 'neutral', 0



In [34]:
skill = EmojiSkill()
text_message = 'Hey bot, are you happy?'
skill.classify(text_message, {})

{'answer': {'confidence_batch': [[1.0]],
  'res_utterances': [['😎']],
  'states': {}},
 'name': 'ExampleSkill'}

In [35]:
skill.classify('The weather is so nice today!) Im charmed', {})

{'answer': {'confidence_batch': [[1.0]],
  'res_utterances': [['🙃']],
  'states': {}},
 'name': 'ExampleSkill'}

In [36]:
skill.classify('I feel so lonely', {})

{'answer': {'confidence_batch': [[0.7]],
  'res_utterances': [['😥']],
  'states': {}},
 'name': 'ExampleSkill'}